In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
import pyspark.sql.types as t
import pyspark.sql.functions as f

In [3]:
schema_mun = 'cod_mun STRING, descricao_mun STRING'
schema_cnae = 'cod_cnae STRING, descricao_cnae STRING'

In [4]:
df_mun = spark.read.csv('gs://igti-mba-de/MUNICIPIOS.csv', sep=';', encoding='ISO-8859-1', schema=schema_mun, escape="\"")
df_cnae = spark.read.csv('gs://igti-mba-de/CNAE.csv', sep=';', encoding='ISO-8859-1', schema=schema_cnae, escape="\"")

In [5]:
df_mun.limit(5).show()

+-------+--------------------+
|cod_mun|       descricao_mun|
+-------+--------------------+
|   0001|       GUAJARA-MIRIM|
|   0002|ALTO ALEGRE DOS P...|
|   0003|         PORTO VELHO|
|   0004|             BURITIS|
|   0005|           JI-PARANA|
+-------+--------------------+



In [6]:
df_cnae.limit(5).show()

+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
| 0111301|    Cultivo de arroz|
| 0111302|    Cultivo de milho|
| 0111303|    Cultivo de trigo|
| 0111399|Cultivo de outros...|
| 0112101|Cultivo de algodã...|
+--------+--------------------+



In [7]:
df_cnae.printSchema()

root
 |-- cod_cnae: string (nullable = true)
 |-- descricao_cnae: string (nullable = true)



In [8]:
df_mun.printSchema()

root
 |-- cod_mun: string (nullable = true)
 |-- descricao_mun: string (nullable = true)



In [9]:
# QUESTÃO 9

cols = ['cnpj', 'cnpj_ordem', 'cnpj_dv', 'id_matriz', 'nome_fantasia',
        'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 
        'nome_cidade', 'cod_pais', 'data_inicio_atividades',
        'cnae_primario', 'cnae_secundario', 
        'tipo_logradouro', 'logradouro', 'numero', 'complemento',
        'bairro', 'cep', 'uf', 'cod_mun', 
        'ddd_1', 'tel_1', 'ddd_2', 'tel_2', 'ddd_fax',
        'correio_eletronico', 'situacao_especial', 'data_situacao_especial']


schema_estab = t.StructType([t.StructField(c, t.StringType()) for c in cols])

df_estab = (
    spark.read
    .format('csv')
    .option('encondig', 'ISO-8859-1')
    .option('sep', ';')
    .option("escape", "\"")
    .schema(schema_estab)
    .load('gs://igti-mba-de/ESTABELECIMENTOS/*')
)

In [10]:
# QUESTÃO 10

df_primario = (
    df_estab.filter('situacao_cadastral = 02')
    .groupby('cnae_primario')
    .count()
    .withColumnRenamed('cnae_primario', 'cod_cnae')
    .orderBy(f.desc('count'))
)


df_secundario = (
    df_estab.filter('situacao_cadastral = 02')
    .withColumn('cod_cnae', f.explode(f.split('cnae_secundario', ',')))
    .groupby('cod_cnae')
    .count()
    .orderBy(f.desc('count'))
)

df_primario.unionByName(df_secundario).groupby('cod_cnae').agg(f.sum('count').alias('count')).orderBy(f.desc('count')).limit(5).toPandas()

,cod_cnae,count
0,4781400,1781558
1,4772500,1021116
2,5611203,937293
3,9602501,893142
4,4723700,854114


In [11]:
# QUESTÃO 11

df_estab.withColumn('data_inicio_atividades', f.to_date(f.col('data_inicio_atividades'), 'yyyyMMdd')).limit(5).toPandas()

21/09/23 15:30:17 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,cnpj,cnpj_ordem,cnpj_dv,id_matriz,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade,cod_pais,...,uf,cod_mun,ddd_1,tel_1,ddd_2,tel_2,ddd_fax,correio_eletronico,situacao_especial,data_situacao_especial
0,26638409,0001,21,1,CANTINHO DA LILICA,02,20161130,00,None,None,...,MG,5425,31,92667948,None,None,None,None,elissouzap@hotmail.com,None
1,26638420,0001,91,1,NOBRE MADEIRAS DO BRASIL,04,20210212,63,None,None,...,GO,9373,62,39881414,None,None,None,None,fmix.contabilidade@gmail.com,None
2,26638434,0001,05,1,TECNODONTOMEDICAL,08,20161216,01,None,None,...,SE,3105,79,98704380,None,None,None,None,alex28ndre@gmail.com,None
3,26638446,0001,30,1,None,04,20210303,63,None,None,...,MA,0849,98,32649607,None,None,None,None,None,None
4,26638459,0001,09,1,None,02,20161130,00,None,None,...,SP,7107,11,99999999,None,None,None,None,None,None


In [12]:
# QUESTÃO 12

df_estab.filter('uf = "SP" and situacao_cadastral != "02"').count()

7966472

In [13]:
# QUESTÃO 13

df_estab.join(df_mun, 'cod_mun')\
.filter('initcap(descricao_mun) = "Belo Horizonte"')\
.filter('(cnae_primario = "6204000" or cnae_secundario like "%6204000%")')\
.count()

6930

In [14]:
# QUESTÃO 14

df_estab.join(df_mun, 'cod_mun').select('nome_fantasia', 'cnae_primario').filter('nome_fantasia like "%IGTI%" and cod_mun = 4123').show(1)

+-------------+-------------+
|nome_fantasia|cnae_primario|
+-------------+-------------+
|         IGTI|      8532500|
+-------------+-------------+



In [16]:
# QUESTÃO 15

df_estab.withColumn('data_inicio_atividades', f.to_date(f.col('data_inicio_atividades'), 'yyyyMMdd')).filter("data_inicio_atividades > date'2019-12-31'").count()

6314456